In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [3]:
from langchain_community.tools import TavilySearchResults

query = "스테이크와 어울리는 와인을 추천해주세요"
web_search = TavilySearchResults(max_results=2)
search_results = web_search.invoke(query)

for result in search_results:
    print(result)
    print("-" * 100)

{'url': 'https://m.blog.naver.com/wineislikeacat/223096696241', 'content': '등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!\n\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\n채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,\n마블링 러버들의 선택을 쉽사리 받는 부위입니다.\n\u200b\n등심과 채끝 스테이크는 모두 풍미가 진하고 지방맛이 강하게 느껴지기에,\n중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\n\u200b\n이에 맞는 레드와인으로는\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\n\u200b\n\n안심 스테이크와 어울리는 와인\n\n안심 스테이크와 어울리는 와인 품종은? 산지오베제!'}
----------------------------------------------------------------------------------------------------
{'url': 'https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/', 'content': 'Bora Kim 2022년 1월 27일\n카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[web_search])

In [5]:
query = "안녕하세요"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls) # 사용할 tools 정보를 보여주는데 여기서는 query가 단순해서 빈리스트가 나옴
print("-" * 100)


AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'stop', 'logprobs': None}, id='run--6a77c7c1-820d-4d15-9f64-364f34017925-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92})
----------------------------------------------------------------------------------------------------
'안녕하세요! 어떻게 도와드릴까요?'
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


In [6]:
query = "스테이크와 어울리는 와인을 추천해주세요"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls) 
print("-" * 100)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wpB0NnWnMscMOxWLjaYxkXaN', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 90, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--312eaa0c-c3f1-4237-9712-84e45dd48bef-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_wpB0NnWnMscMOxWLjaYxkXaN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 27, 'total_tokens': 117})
-------------------------------------------------

In [7]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_wpB0NnWnMscMOxWLjaYxkXaN',
 'type': 'tool_call'}

In [8]:
tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출결과:")
print("-" * 100)
print(tool_output)

tavily_search_results_json 호출결과:
----------------------------------------------------------------------------------------------------
[{'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '리브아이 스테이크; 뉴욕 스트립; 티본 스테이크. 추천 와인: 나파 밸리 카베르네 소비뇽; 샤또 마고 (Château Margaux). 카베르네 소비뇽은 특히 마블링'}, {'url': 'https://m.blog.naver.com/wineislikeacat/223096696241', 'content': '등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!\n\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\n채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,\n마블링 러버들의 선택을 쉽사리 받는 부위입니다.\n\u200b\n등심과 채끝 스테이크는 모두 풍미가 진하고 지방맛이 강하게 느껴지기에,\n중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\n\u200b\n이에 맞는 레드와인으로는\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\n\u200b\n\n안심 스테이크와 어울리는 와인\n\n안심 스테이크와 어울리는 와인 품종은? 산지오베제

In [9]:
from langchain_core.messages import ToolMessage
tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

print(tool_message)

content=[{'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '리브아이 스테이크; 뉴욕 스트립; 티본 스테이크. 추천 와인: 나파 밸리 카베르네 소비뇽; 샤또 마고 (Château Margaux). 카베르네 소비뇽은 특히 마블링'}, {'url': 'https://m.blog.naver.com/wineislikeacat/223096696241', 'content': '등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!\n\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\n채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,\n마블링 러버들의 선택을 쉽사리 받는 부위입니다.\n\u200b\n등심과 채끝 스테이크는 모두 풍미가 진하고 지방맛이 강하게 느껴지기에,\n중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\n\u200b\n이에 맞는 레드와인으로는\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\n\u200b\n\n안심 스테이크와 어울리는 와인\n\n안심 스테이크와 어울리는 와인 품종은? 산지오베제!'}] name='tavily_search_results_json' tool_call_id='call_wpB0NnWnMscMOxWLjaYxkXaN'


In [10]:
tool_message = web_search.invoke(tool_call)
print(tool_message)

content='[{"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "리브아이 스테이크; 뉴욕 스트립; 티본 스테이크. 추천 와인: 나파 밸리 카베르네 소비뇽; 샤또 마고 (Château Margaux). 카베르네 소비뇽은 특히 마블링"}, {"url": "https://m.blog.naver.com/wineislikeacat/223096696241", "content": "등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!\\n\\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\\n채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,\\n마블링 러버들의 선택을 쉽사리 받는 부위입니다.\\n\u200b\\n등심과 채끝 스테이크는 모두 풍미가 진하고 지방맛이 강하게 느껴지기에,\\n중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\\n\u200b\\n이에 맞는 레드와인으로는\\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\\n\u200b\\n\\n안심 스테이크와 어울리는 와인\\n\\n안심 스테이크와 어울리는 와인 품종은? 산지오베제!"}]' name='tavily_search_results_json' tool_call_id='call_wpB0NnWnMscMOxWLjaYxkXaN' artifact={'query': '스테이

In [11]:
pprint(tool_message.content)

('[{"url": '
 '"https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", '
 '"content": "리브아이 스테이크; 뉴욕 스트립; 티본 스테이크. 추천 와인: 나파 밸리 카베르네 소비뇽; 샤또 마고 '
 '(Château Margaux). 카베르네 소비뇽은 특히 마블링"}, {"url": '
 '"https://m.blog.naver.com/wineislikeacat/223096696241", "content": "등심 스테이크, '
 '채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!\\n\\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\\n육질이 '
 '연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\\n채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,\\n마블링 '
 '러버들의 선택을 쉽사리 받는 부위입니다.\\n\u200b\\n등심과 채끝 스테이크는 모두 풍미가 진하고 지방맛이 강하게 '
 '느껴지기에,\\n중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 '
 '있습니다.\\n\u200b\\n이에 맞는 레드와인으로는\\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) '
 '품종을 추천드려요!\\n\u200b\\n\\n안심 스테이크와 어울리는 와인\\n\\n안심 스테이크와 어울리는 와인 품종은? '
 '산지오베제!"}]')


In [12]:
tool_message = web_search.batch(ai_msg.tool_calls)
pprint(tool_message)
print("-" * 100)
pprint(tool_message[0].content)

[ToolMessage(content='[{"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "리브아이 스테이크; 뉴욕 스트립; 티본 스테이크. 추천 와인: 나파 밸리 카베르네 소비뇽; 샤또 마고 (Château Margaux). 카베르네 소비뇽은 특히 마블링"}, {"url": "https://m.blog.naver.com/wineislikeacat/223096696241", "content": "등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!\\n\\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\\n채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,\\n마블링 러버들의 선택을 쉽사리 받는 부위입니다.\\n\u200b\\n등심과 채끝 스테이크는 모두 풍미가 진하고 지방맛이 강하게 느껴지기에,\\n중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\\n\u200b\\n이에 맞는 레드와인으로는\\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\\n\u200b\\n\\n안심 스테이크와 어울리는 와인\\n\\n안심 스테이크와 어울리는 와인 품종은? 산지오베제!"}]', name='tavily_search_results_json', tool_call_id='call_wpB0NnWnMscMOxWLjaYxkXaN', artifac

In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.today().strftime("%Y-%m-%d")

prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant, Today's date is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[web_search])

llm_chain = prompt | llm_with_tools

@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-" * 100)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-" * 100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 가격은 얼마인가요?")

pprint(response.content)


ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_d15baCX4PI9bP6Hc1ZH6ebFF', 'function': {'arguments': '{"query":"모엣샹동 샴페인 가격 2025년 5월"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 114, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--528ca666-196a-4a25-aae9-5066245a5c65-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '모엣샹동 샴페인 가격 2025년 5월'}, 'id': 'call_d15baCX4PI9bP6Hc1ZH6ebFF', 'type': 'tool_call'}] usage_metadata={'input_tokens': 114, 'output_tokens': 35, 'total_tokens': 149}
---------------------------------------------

In [14]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

@tool
def search_web(query: str) -> List[str]:
    """ tool 용도 기재 """
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [15]:
print# 도구 속성
print("자료형: ")
print(type(search_web))
print("-"*100)

print("name: ")
print(search_web.name)
print("-"*100)

print("description: ")
pprint(search_web.description)
print("-"*100)

print("schema: ")
pprint(search_web.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_web
----------------------------------------------------------------------------------------------------
description: 
'tool 용도 기재'
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'tool 용도 기재 ',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_web',
 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [16]:
query = "스테이크와 어울리는 와인을 추천해주세요."
search_results = search_web.invoke(query)

print(search_results)

<Document href="https://m.blog.naver.com/wineislikeacat/223096696241"/>
등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!

등심은 소의 등뼈를 둘러싸고 있는 부위입니다.
육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.
채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,
마블링 러버들의 선택을 쉽사리 받는 부위입니다.
​
등심과 채끝 스테이크는 모두 풍미가 진하고 지방맛이 강하게 느껴지기에,
중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면
씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.
​
이에 맞는 레드와인으로는
카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!
​

안심 스테이크와 어울리는 와인

안심 스테이크와 어울리는 와인 품종은? 산지오베제!
</Document>
---
<Document href="https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/"/>
Bora Kim 2022년 1월 27일
카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.
</Document>


In [17]:
llm_with_tools = llm.bind_tools(tools=[search_web])
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YhUvqqGs45vLZUSJx3b5lBj4', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 55, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_92e0377081', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ad7bbf49-30f7-49bd-9627-fb66f344f4dd-0', tool_calls=[{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_YhUvqqGs45vLZUSJx3b5lBj4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 23, 'total_tokens': 78})
-----------------------------------------------------------------------------------